In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_recipes = pd.read_csv('/content/gdrive/MyDrive/Data/RAW_recipes.csv')
df_user_interaction= pd.read_csv('/content/gdrive/MyDrive/Data/RAW_interactions.csv')

In [ ]:
df_recipes.head(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
df_recipes.isna().sum()

name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64

In [ ]:
df_recipes.dropna(subset = ['name'], inplace=True)
df_recipes.isna().sum()

name                 0
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64

In [ ]:
df_recipes.iloc[0, 8]

"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']"

In [ ]:
import ast
df_recipes['tags']= [ast.literal_eval(x) for x in df_recipes['tags']]
df_recipes['ingredients']= [ast.literal_eval(x) for x in df_recipes['ingredients']]

In [ ]:
df_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8


In [ ]:
df_recipes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231636 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231636 non-null  int64 
 2   minutes         231636 non-null  int64 
 3   contributor_id  231636 non-null  int64 
 4   submitted       231636 non-null  object
 5   tags            231636 non-null  object
 6   nutrition       231636 non-null  object
 7   n_steps         231636 non-null  int64 
 8   steps           231636 non-null  object
 9   description     226657 non-null  object
 10  ingredients     231636 non-null  object
 11  n_ingredients   231636 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 23.0+ MB


In [ ]:
nd_array = np.array(df_recipes["name"])
type(nd_array[0])

str

In [ ]:
nd_tags = np.asarray(df_recipes.tags)
nd_ingredients = np.asarray(df_recipes.ingredients)

In [ ]:
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
ingredients_all = df_recipes.ingredients.apply(lambda x: ", ".join(x))

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b')

# Fit and transform
X = vect.fit_transform(ingredients_all)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics = 25,passes = 5, random_state = 0, id2word = id_map)

In [ ]:
dir(ldamodel)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_apply',
 '_load_specials',
 '_save_specials',
 '_smart_save',
 'alpha',
 'bound',
 'callbacks',
 'chunksize',
 'clear',
 'decay',
 'diff',
 'dispatcher',
 'distributed',
 'do_estep',
 'do_mstep',
 'dtype',
 'eta',
 'eval_every',
 'expElogbeta',
 'gamma_threshold',
 'get_document_topics',
 'get_term_topics',
 'get_topic_terms',
 'get_topics',
 'id2word',
 'inference',
 'init_dir_prior',
 'iterations',
 'load',
 'log_perplexity',
 'minimum_phi_value',
 'minimum_probability',
 'num_terms',
 'num_topics',
 'num_updates',
 'numworkers',
 'offset',
 'optimize_alpha',
 'optimize_eta',
 'pas

In [ ]:
ldamodel

In [ ]:
ldamodel.print_topics(num_topics = 25,num_words = 5)

[(0,
  '0.076*"sauce" + 0.063*"pepper" + 0.056*"ground" + 0.052*"onion" + 0.050*"beef" + 0.046*"powder" + 0.044*"garlic" + 0.039*"salt" + 0.038*"tomato" + 0.029*"beans" + 0.025*"worcestershire" + 0.021*"chili" + 0.018*"water" + 0.017*"black" + 0.017*"tomatoes"'),
 (1,
  '0.075*"pepper" + 0.047*"fresh" + 0.044*"salt" + 0.042*"garlic" + 0.040*"oil" + 0.040*"red" + 0.039*"ground" + 0.032*"powder" + 0.031*"onion" + 0.030*"cilantro" + 0.029*"cumin" + 0.028*"bell" + 0.023*"green" + 0.022*"black" + 0.020*"chili"'),
 (2,
  '0.105*"baking" + 0.092*"sugar" + 0.082*"flour" + 0.065*"powder" + 0.059*"salt" + 0.048*"vanilla" + 0.047*"soda" + 0.042*"eggs" + 0.040*"butter" + 0.027*"all" + 0.026*"purpose" + 0.026*"extract" + 0.024*"brown" + 0.023*"egg" + 0.022*"milk"'),
 (3,
  '0.167*"syrup" + 0.096*"maple" + 0.088*"nutmeg" + 0.080*"molasses" + 0.057*"toasted" + 0.053*"watermelon" + 0.039*"clove" + 0.025*"cut" + 0.023*"sourdough" + 0.023*"flavor" + 0.021*"crisco" + 0.021*"pure" + 0.020*"flavoring" + 0.

In [ ]:
def Sort_Tuple(tup):  
    return(sorted(tup, key = lambda x: x[1], reverse = True))   

In [ ]:
doc_num, topic_num, prob = [], [], []
print(ldamodel.get_document_topics(corpus))
for n in range(len(df_recipes)):
 get_document_topics = ldamodel.get_document_topics(corpus[n])
 doc_num.append(n)
 sorted_doc_topics = Sort_Tuple(get_document_topics)
 topic_num.append(sorted_doc_topics[0][0])
 prob.append(sorted_doc_topics[0][1])
df_recipes["Doc"] = doc_num
df_recipes["Topic"] = topic_num
df_recipes["Probability"] = prob
df_recipes.to_csv("doc_topic_matrix.csv", index=False)

In [ ]:
df_recipes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231636 entries, 0 to 231636
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            231636 non-null  string 
 1   id              231636 non-null  int64  
 2   minutes         231636 non-null  int64  
 3   contributor_id  231636 non-null  int64  
 4   submitted       231636 non-null  object 
 5   tags            231636 non-null  object 
 6   nutrition       231636 non-null  object 
 7   n_steps         231636 non-null  int64  
 8   steps           231636 non-null  object 
 9   description     226657 non-null  object 
 10  ingredients     231636 non-null  object 
 11  n_ingredients   231636 non-null  int64  
 12  Doc             231636 non-null  int64  
 13  Topic           231636 non-null  int64  
 14  Probababilty    231636 non-null  float64
 15  Probabilty      231636 non-null  float64
dtypes: float64(2), int64(7), object(6), string(1)
memory usa

In [ ]:
df_recipes["name"] = df_recipes["name"].astype("string")

In [ ]:
def recommend_by_ingredients(name, df):
 recommended = []
 top10_list = []
 name = name.replace(" ","")
 topic_num = df_recipes[(df_recipes["name"].str.replace(" ",""))==name].Topic.values
 doc_num = df_recipes[(df_recipes["name"].str.replace(" ",""))==name].Doc.values 
 output_df = df_recipes[df_recipes["Topic"]==topic_num[0]].sort_values("Probability", ascending=False).reset_index(drop=True)
 index = output_df[output_df["Doc"]==doc_num[0]].index[0]
 
 top10_list += list(output_df.iloc[index-5:index].index)
 top10_list += list(output_df.iloc[index+1:index+6].index)
 
 output_df["name"] = output_df["name"].str.title()
 for each in top10_list:
    recommended.append(output_df.iloc[each]["name"])
 return recommended
 

In [ ]:
recommend_by_ingredients("arriba baked winter squash mexican style", df_recipes)

['Mu Shu Turkey Wraps',
 'Crock Pot Pork Stroganoff',
 'Joy S Chicken Pot Pies',
 'Zuppa Toscana Soup   Light Version',
 'Turkey Corn Chowder',
 'Sweet Peas And Carrots In A Tomato Sauce',
 'Rose A  Rita  Cocktail',
 'Chicken  N  Cheese Casserole',
 'Baked Potato Salad   Loaded',
 'Baked Cheesy Chicken Spaghetti']